# Standard Star Photometry and Color Analysis

This notebook demonstrates:
1. Photometric calibration using standard stars
2. Color index calculations (e.g., B-V, V-R)
3. Zero point determination
4. Transformation to standard system

## Prerequisites
- Observations of standard stars in multiple filters
- Known magnitudes from catalog (e.g., Landolt standards)

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path

from src.data_reduction import load_fits
from src.photometry import (
    find_sources,
    measure_aperture_photometry,
    calculate_color_index
)
from src.utils import list_fits_files, plot_image

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

## 1. Load Standard Star Observations

Organize your standard star observations by filter.

In [ ]:
# Define filter names and corresponding file patterns
filters = ['B', 'V', 'R']  # Adjust for your observations

# Directory containing standard star observations
standards_dir = Path('../data/reduced')

# Load observations (adjust patterns to match your files)
filter_files = {}
for filt in filters:
    pattern = f'*{filt}*.fits'  # Adjust pattern as needed
    files = list_fits_files(standards_dir, pattern=pattern)
    filter_files[filt] = files
    print(f"Filter {filt}: {len(files)} files")

## 2. Define Standard Star Catalog

Create a catalog of known standard stars and their magnitudes.

In [ ]:
# Example: Landolt standard star data
# Replace with actual catalog values for your field

standard_catalog = pd.DataFrame({
    'star_id': ['SA98-193', 'SA98-194', 'SA98-195'],
    'ra': [21.0, 21.1, 21.2],  # Right Ascension (deg)
    'dec': [0.0, 0.1, 0.2],    # Declination (deg)
    'V_mag': [13.150, 13.560, 14.020],  # Johnson V magnitude
    'B_mag': [13.850, 14.320, 14.880],  # Johnson B magnitude
    'R_mag': [12.700, 13.050, 13.450],  # Cousins R magnitude
    'x_pixel': [512.5, 612.3, 412.8],   # Approximate pixel positions
    'y_pixel': [512.5, 612.7, 412.2]
})

# Calculate known color indices
standard_catalog['B-V'] = standard_catalog['B_mag'] - standard_catalog['V_mag']
standard_catalog['V-R'] = standard_catalog['V_mag'] - standard_catalog['R_mag']

print("Standard Star Catalog:")
print(standard_catalog)

## 3. Measure Instrumental Magnitudes

Perform photometry on standard stars in each filter.

In [ ]:
# Store instrumental magnitudes
instrumental_mags = {filt: [] for filt in filters}

aperture_radius = 10  # Adjust based on seeing

for filt in filters:
    if len(filter_files[filt]) > 0:
        # Use first file for each filter (or average multiple)
        data, header = load_fits(filter_files[filt][0])
        
        # Get standard star positions
        positions = list(zip(standard_catalog['x_pixel'], 
                           standard_catalog['y_pixel']))
        
        # Measure photometry
        phot = measure_aperture_photometry(
            data, positions, aperture_radius=aperture_radius
        )
        
        instrumental_mags[filt] = phot['mag_inst']
        
        print(f"\n{filt}-band instrumental magnitudes:")
        for i, mag in enumerate(phot['mag_inst']):
            print(f"  {standard_catalog['star_id'].iloc[i]}: {mag:.3f}")

## 4. Calculate Zero Points

Determine photometric zero points for each filter.

In [ ]:
zero_points = {}

for filt in filters:
    if len(instrumental_mags[filt]) > 0:
        # Zero point: standard_mag = instrumental_mag + ZP
        catalog_mags = standard_catalog[f'{filt}_mag'].values
        inst_mags = instrumental_mags[filt]
        
        zp_values = catalog_mags - inst_mags
        zero_point = np.mean(zp_values)
        zero_points[filt] = zero_point
        
        print(f"\n{filt}-band zero point: {zero_point:.3f} ± {np.std(zp_values):.3f}")
        print(f"Individual ZP values: {zp_values}")

## 5. Plot Color-Magnitude Diagram

In [ ]:
# Plot B-V color vs V magnitude
if 'B' in zero_points and 'V' in zero_points:
    fig, ax = plt.subplots(figsize=(10, 8))
    
    ax.scatter(standard_catalog['B-V'], standard_catalog['V_mag'], 
              s=100, alpha=0.7, edgecolors='black', linewidths=2)
    
    # Label stars
    for _, star in standard_catalog.iterrows():
        ax.annotate(star['star_id'], 
                   (star['B-V'], star['V_mag']),
                   xytext=(5, 5), textcoords='offset points',
                   fontsize=10)
    
    ax.invert_yaxis()  # Brighter stars at top
    ax.set_xlabel('B - V (mag)', fontsize=12)
    ax.set_ylabel('V (mag)', fontsize=12)
    ax.set_title('Color-Magnitude Diagram', fontsize=14)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('../data/color_magnitude_diagram.png', dpi=150, bbox_inches='tight')
    plt.show()

## 6. Calculate Color Indices

In [ ]:
# Calculate instrumental color indices
if 'B' in instrumental_mags and 'V' in instrumental_mags:
    inst_BV = instrumental_mags['B'] - instrumental_mags['V']
    catalog_BV = standard_catalog['B-V'].values
    
    # Plot instrumental vs standard colors
    fig, ax = plt.subplots(figsize=(10, 8))
    
    ax.scatter(catalog_BV, inst_BV, s=100, alpha=0.7)
    
    # Fit linear transformation
    coeffs = np.polyfit(catalog_BV, inst_BV, 1)
    fit_line = np.poly1d(coeffs)
    x_fit = np.linspace(catalog_BV.min(), catalog_BV.max(), 100)
    ax.plot(x_fit, fit_line(x_fit), 'r--', 
           label=f'Fit: y = {coeffs[0]:.3f}x + {coeffs[1]:.3f}')
    
    ax.set_xlabel('Standard (B-V)', fontsize=12)
    ax.set_ylabel('Instrumental (B-V)', fontsize=12)
    ax.set_title('Color Transformation', fontsize=14)
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('../data/color_transformation.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"\nColor transformation:")
    print(f"  (B-V)_inst = {coeffs[0]:.3f} × (B-V)_std + {coeffs[1]:.3f}")

## 7. Export Calibration Results

In [ ]:
# Save zero points and transformations
calibration_data = {
    'filter': list(zero_points.keys()),
    'zero_point': list(zero_points.values())
}

calib_df = pd.DataFrame(calibration_data)
calib_df.to_csv('../data/photometric_calibration.csv', index=False)

print("Photometric Calibration Results:")
print(calib_df)
print("\nSaved to ../data/photometric_calibration.csv")

## 8. Apply Calibration to Science Target

Use the derived zero points to calibrate your science observations.

In [ ]:
# Example: calibrate target magnitudes
# Replace with your actual target photometry

if 'V' in zero_points:
    # Example instrumental magnitude from your target
    target_inst_mag_V = -15.5  # Replace with actual value
    
    # Apply zero point
    target_calibrated_V = target_inst_mag_V + zero_points['V']
    
    print(f"\nTarget Calibration:")
    print(f"  Instrumental V: {target_inst_mag_V:.3f}")
    print(f"  Calibrated V: {target_calibrated_V:.3f}")

## Summary

✓ Standard star photometry completed
✓ Zero points calculated for each filter
✓ Color transformations derived
✓ Calibration data saved

**Next Steps:**
- Apply calibration to science targets
- Calculate final calibrated magnitudes and colors
- Prepare figures and tables for manuscript
- Export to Overleaf